In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_05_02'
posner_id ='pos2a'
posner_id_b ='pos2b'

IMPORTED MODULES today: 2023-07-17 20:57:26.471786


In [4]:
%matplotlib qt

# Posner A

In [5]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_05_02/m_05_02_pos2a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_05_02/05_2a_posner_task_2023-05-17_15h16.18.329/05_2a_posner_task_2023-05-17_15h16.18.329.csv
Found '2b' in file: m_05_02/05_2b_posner_task_2023-05-17_16h14.34.260/05_2b_posner_task_2023-05-17_16h14.34.260.csv
********** 2a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-05-17 15:20:01.470839+00:00 - 2023-05-17 15:32:13.510839+00:00
  [0] Keeping  (2023-05-17 15:20:01.470839+00:00 - 2023-05-17 15:20:01.470839+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-17 15:20:39.100839+00:00 - 2023-05-17 15:20:39.100839+00:00 -> 37.63 - 37.63)
  [2] Keeping  (2023-05-17 15:20:40.600839+00:00 - 2023-05-17 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


In [6]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
411 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 411 events and 701 original time points ...
3 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [7]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 37.8s.


Method,fastica
Fit,59 iterations on raw data (681643 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [8]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=681643
    Range : 37630 ... 719272 =     37.630 ...   719.272 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated


In [9]:
print(ica.exclude)

[0, 1, 4, 17, 21, 24, 27, 28, 26, 37, 46, 49, 45]


In [12]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 13 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [14]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1, overwrite=True)

m_05_02_pos2a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_05_02

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_05_02/m_05_02_pos2a.vhdr
Overwriting existing file.


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [15]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_05_02
├── m_05_02_pos2a.eeg
├── m_05_02_pos2a.vhdr
├── m_05_02_pos2a.vmrk
├── m_05_02_pos2b.eeg
├── m_05_02_pos2b.vhdr
└── m_05_02_pos2b.vmrk

1 directory, 6 files


In [16]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [12]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_05_02/m_05_02_pos2b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_05_02/05_2a_posner_task_2023-05-17_15h16.18.329/05_2a_posner_task_2023-05-17_15h16.18.329.csv
Found '2b' in file: m_05_02/05_2b_posner_task_2023-05-17_16h14.34.260/05_2b_posner_task_2023-05-17_16h14.34.260.csv
********** 2b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-05-17 16:17:53.150185+00:00 - 2023-05-17 16:31:16.710185+00:00
  [0] Keeping  (2023-05-17 16:17:53.150185+00:00 - 2023-05-17 16:17:53.150185+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-17 16:19:18.926185+00:00 - 2023-05-17 16:19:18.926185+00:00 -> 85.776 - 85.776)
  [2] Keeping  (2023-05-17 16:19:20.426185+00:00 - 2023-05-17

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [215] Keeping  (2023-05-17 16:25:42.118185+00:00 - 2023-05-17 16:25:42.118185+00:00 -> 468.968 - 468.968)
  [216] Keeping  (2023-05-17 16:25:43.618185+00:00 - 2023-05-17 16:25:43.618185+00:00 -> 470.468 - 470.468)
  [217] Keeping  (2023-05-17 16:25:47.038185+00:00 - 2023-05-17 16:25:47.038185+00:00 -> 473.888 - 473.888)
  [218] Keeping  (2023-05-17 16:25:47.138185+00:00 - 2023-05-17 16:25:47.138185+00:00 -> 473.988 - 473.988)
  [219] Keeping  (2023-05-17 16:25:48.618185+00:00 - 2023-05-17 16:25:48.618185+00:00 -> 475.468 - 475.468)
  [220] Keeping  (2023-05-17 16:25:50.118185+00:00 - 2023-05-17 16:25:50.118185+00:00 -> 476.968 - 476.968)
  [221] Keeping  (2023-05-17 16:25:52.778185+00:00 - 2023-05-17 16:25:52.778185+00:00 -> 479.628 - 479.628)
  [222] Keeping  (2023-05-17 16:25:52.878185+00:00 - 2023-05-17 16:25:52.878185+00:00 -> 479.728 - 479.728)
  [223] Keeping  (2023-05-17 16:25:55.118185+00:00 - 2023-05-17 16:25:55.118185+00:00 -> 481.968 - 481.968)
  [224] Keeping  (2023-05-17

In [13]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
411 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 411 events and 701 original time points ...
3 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.8s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [14]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-05-17 16:19:18.926185+00:00 - 2023-05-17 16:31:05.778185+00:00
  [0] Keeping  (2023-05-17 16:19:18.926185+00:00 - 2023-05-17 16:19:18.926185+00:00 -> 85.776 - 85.776)
  [1] Keeping  (2023-05-17 16:19:20.426185+00:00 - 2023-05-17 16:19:20.426185+00:00 -> 87.276 - 87.276)
  [2] Keeping  (2023-05-17 16:19:23.525185+00:00 - 2023-05-17 16:19:23.525185+00:00 -> 90.375 - 90.375)
  [3] Keeping  (2023-05-17 16:19:23.625185+00:00 - 2023-05-17 16:19:23.625185+00:00 -> 90.475 - 90.475)
  [4] Keeping  (2023-05-17 16:19:25.426185+00:00 - 2023-05-17 16:19:25.426185+00:00 -> 92.276 - 92.276)
  [5] Keeping  (2023-05-17 16:19:26.926185+00:00 - 2023-05-17 16:19:26.926185+00:00 -> 93.776 - 93.776)
  [6] Keeping  (2023-05-17 16:19:30.275185+00:00 - 2023-05-17 16:19:30.275185+00:00 -> 97.125 - 97.125)
  [7] Keeping  (2023-05-17 16:19:30.375185+00:00 - 2023-05-17 16:19

Method,fastica
Fit,58 iterations on raw data (706852 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [15]:
ica_b.plot_components()

[<MNEFigure size 1950x1832 with 20 Axes>,
 <MNEFigure size 1950x1832 with 20 Axes>,
 <MNEFigure size 1950x1832 with 20 Axes>,
 <MNEFigure size 1170x520 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=706852
    Range : 85776 ... 792627 =     85.776 ...   792.627 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
353 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
353 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
353 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
353 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
353 matching events found
No baseline correction applied
0 projection items activated


In [16]:
ica_b.exclude

[1, 0, 2, 4, 15, 30, 31, 36, 32, 29, 28, 27, 43, 42]

In [17]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 14 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [21]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_05_02/m_05_02_pos2b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [22]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_05_02
├── m_05_02_pos2a.eeg
├── m_05_02_pos2a.vhdr
├── m_05_02_pos2a.vmrk
├── m_05_02_pos2b.eeg
├── m_05_02_pos2b.vhdr
└── m_05_02_pos2b.vmrk

1 directory, 6 files


# ...

In [23]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b

: 